In [2]:
from langchain_aws import ChatBedrock

## LangSmith 설정
LangSmith를 통해 애플리케이션 내부 단계를 추적하고 분석 가능

In [3]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")

## 언어 모델 사용

In [4]:
from langchain_aws import ChatBedrock  # Bedrock 모델 사용
from langchain_core.messages import HumanMessage, SystemMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0),
    region_name='us-east-1'
)

# 영어에서 이탈리아어로 번역하는 예시
messages = [
    SystemMessage(content="Translate the following from English into Korean"),
    HumanMessage(content="hi!"),
]

result = model.invoke(messages)
print(result)


content='안녕하세요!' additional_kwargs={'usage': {'prompt_tokens': 17, 'completion_tokens': 11, 'total_tokens': 28}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} response_metadata={'usage': {'prompt_tokens': 17, 'completion_tokens': 11, 'total_tokens': 28}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} id='run-26e76738-22ed-4916-b980-606eaf0054ce-0' usage_metadata={'input_tokens': 17, 'output_tokens': 11, 'total_tokens': 28}


In [5]:
messages

[SystemMessage(content='Translate the following from English into Korean', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [9]:
# result = model.invoke('LLMOps에 대해서 소개해줘')
result = model.invoke('너는 영어로 ')
result

## OutputParser 사용
OutputParser를 사용하여 결과의 텍스트만 추출할 수 있음

In [10]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 설정
parser = StrOutputParser()

# 모델 결과를 파서로 처리
parsed_result = parser.invoke(result)
print(parsed_result)  # 출력: '안녕!'


LLMOps(Large Language Model Operations)는 대규모 언어 모델(LLM)의 개발, 배포, 유지보수 및 모니터링을 위한 일련의 실무와 프로세스를 말합니다. 이는 MLOps(Machine Learning Operations)의 개념을 LLM에 특화시킨 것으로 볼 수 있습니다.

LLMOps의 주요 특징과 구성요소는 다음과 같습니다:

1. 모델 개발 및 훈련:
   - 데이터 수집 및 전처리
   - 모델 아키텍처 설계
   - 하이퍼파라미터 튜닝
   - 분산 학습 및 최적화

2. 배포 및 서빙:
   - 모델 압축 및 최적화
   - 컨테이너화 및 오케스트레이션
   - API 개발 및 관리
   - 스케일링 및 로드 밸런싱

3. 모니터링 및 유지보수:
   - 성능 모니터링
   - 데이터 드리프트 감지
   - 모델 업데이트 및 재훈련
   - 버전 관리

4. 보안 및 규정 준수:
   - 데이터 프라이버시 보호
   - 편향성 감지 및 완화
   - 윤리적 AI 사용 보장
   - 규제 준수

5. 인프라 관리:
   - 클라우드 리소스 최적화
   - 비용 관리
   - 고가용성 및 재해 복구

6. 협업 및 워크플로우:
   - 버전 제어 시스템 통합
   - CI/CD 파이프라인 구축
   - 팀 협업 도구 활용

7. 실험 및 평가:
   - A/B 테스팅
   - 성능 벤치마킹
   - 사용자 피드백 수집 및 분석

LLMOps는 LLM의 복잡성과 규모로 인해 전통적인 MLOps보다 더 많은 도전과제를 안고 있습니다. 예를 들어, 모델 크기로 인한 컴퓨팅 자원 관리, 지속적인 학습 및 개선, 윤리적 고려사항 등이 있습니다.

효과적인 LLMOps 구현은 LLM 프로젝트의 성공적인 운영과 지속 가능한 발전을 위해 필수적입니다. 이를 통해 조직은 LLM의 잠재력을 최대한 활용하면서도 안정성, 확장성, 그리고 책임 있는 AI 사용을 보장할 수 있습니다.


### 체인으로 모델과 파서 결합

In [11]:
# # model, parser를 한번에 invoke
# vars = [model, parser]
# for var in vars:
#     var.invoke()

In [12]:
chain = model | parser

In [14]:
parsed_result = chain.invoke(messages)
parsed_result

'안녕하세요!'

### 프롬프트 템플릿 사용

In [15]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지 템플릿 생성
system_template = "Translate the following into {language}:"

# 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# 템플릿 호출
prompt_result = prompt_template.invoke({"language": "korean", "text": "hi"})
print(prompt_result.to_messages())


[SystemMessage(content='Translate the following into korean:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]


In [16]:
# 프롬프트, 모델, 파서를 결합한 체인 생성
chain = prompt_template | model | parser

# 체인 호출
result = chain.invoke({"language": "korean", "text": "hi"})
print(result)  # 출력: 'ciao'


안녕하세요 (annyeonghaseyo)

This is a common Korean greeting that can be used in both formal and informal situations. It's equivalent to "hello" in English.


### 서버 파일 작성